In [ ]:
import pandas as pd
import json

# Load data utama (dataset risiko gizi)
df_risiko = pd.read_csv("dataset.csv")

# Load file JSON
with open("regencies.json", "r", encoding="utf-8") as file:
    data_json = json.load(file)

# Ubah JSON menjadi DataFrame
df_json = pd.DataFrame(data_json)

# Pastikan kolom kode sama-sama string
df_risiko['Kode_Kabupaten_Kota'] = df_risiko['Kode_Kabupaten_Kota'].astype(str)
df_json['id'] = df_json['id'].astype(str)

# Gabungkan berdasarkan kode kabupaten/kota
df_gabungan = pd.merge(
    df_risiko,
    df_json[['id', 'latitude', 'longitude']],
    left_on='Kode_Kabupaten_Kota',
    right_on='id',
    how='left'
)

# Tampilkan beberapa baris hasil gabungan
print(df_gabungan.head())

# Simpan ke file CSV baru
df_gabungan.to_csv("data_gabungan_dengan_koordinat.csv", index=False)


In [1]:
import pandas as pd

# Baca data kamu yang sudah tergabung dengan koordinat
df = pd.read_csv("data_gabungan_dengan_koordinat.csv")  # ganti jika nama file berbeda

# Tambahkan kolom Risiko_Gizi berdasarkan PoU
def klasifikasi_risiko(pou):
    if pou < 5:
        return 'Rendah'
    elif 5 <= pou < 20:
        return 'Sedang'
    else:
        return 'Tinggi'

df['Risiko_Gizi'] = df['PoU'].apply(klasifikasi_risiko)

# Simpan ke CSV
df.to_csv("data_final_dengan_risiko_gizi_baru.csv", index=False)

print("Kolom 'Risiko_Gizi' berhasil ditambahkan dan disimpan ke file CSV baru.")


Kolom 'Risiko_Gizi' berhasil ditambahkan dan disimpan ke file CSV baru.


In [ ]:
import folium
import pandas as pd
from branca.element import Template, MacroElement

# Baca data
df = pd.read_csv("data_final_dengan_prediksi_aljabar.csv")

# Buat peta dasar (tengah-tengah Indonesia)
peta = folium.Map(location=[-2.5, 118], zoom_start=5)

# Warna berdasarkan risiko
warna_risiko = {
    'Rendah': 'green',
    'Sedang': 'orange',
    'Tinggi': 'red'
}

# Tambahkan titik satu per satu berdasarkan risiko gizi
for _, row in df.iterrows():
    if pd.notnull(row['latitude']) and pd.notnull(row['longitude']):
        risiko = row['Risiko_Gizi']
        warna = warna_risiko.get(risiko, 'gray')  # default warna abu kalau tidak ada
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=6,
            color=warna,
            fill=True,
            fill_opacity=0.7,
            popup=f"{row['Kabupaten_Kota']}<br>Risiko: {risiko}"
        ).add_to(peta)

# Tambahkan legenda manual
template = """
{% macro html(this, kwargs) %}
<div style="
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 150px;
    height: 110px;
    z-index:9999;
    background-color: white;
    border:2px solid grey;
    padding: 10px;
    font-size:14px;
">
    <b>Legenda:</b><br>
    <i style="background:green; width:10px; height:10px; display:inline-block;"></i> Rendah<br>
    <i style="background:orange; width:10px; height:10px; display:inline-block;"></i> Sedang<br>
    <i style="background:red; width:10px; height:10px; display:inline-block;"></i> Tinggi<br>
</div>
{% endmacro %}
"""

legend = MacroElement()
legend._template = Template(template)
peta.get_root().add_child(legend)

# Simpan peta sebagai HTML
peta.save("peta_risiko_gizi_interaktif_aljabar.html")
